In [2]:
# !pip install unsloth
# !pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.7 MB/s eta 0:00:00
ERROR: Operation cancelled by user
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-jv7bqkva
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-jv7bqkva
  Resolved https://github.com/unslothai/unsloth.git to commit 5b64813ffb83d6e507989410a64d3b64fbffb323
  Installing build dependencies ... canceled
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
   

In [3]:
pip install unsloth unsloth_zoo --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 132.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 150.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.7/405.7 kB 266.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 199.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 282.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 182.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.3/566.3 kB 240.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 132.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8

In [4]:
from unsloth import FastLanguageModel
import torch
from transformers import TrainingArguments
import wandb
from datasets import load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
from huggingface_hub import login
from google.colab import userdata

In [6]:
hf_token = userdata.get("HF_TOKEN")
login(hf_token)

In [7]:
#Loading the dataset

In [8]:
from itertools import islice
dataset_stream = load_dataset('Prarabdha/indian-legal-supervised-fine-tuning-data', split = "train", streaming = True)
law_dataset = list(islice(dataset_stream, 1000))

README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/33 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/33 [00:00<?, ?it/s]

In [9]:
len(law_dataset)

1000

In [34]:
law_dataset[12]

{'context': 'The next question is whether these activities were carried on by a D\ncongregation of Sadhus at the site and not by the State Govt. and despite\nGovernments efforts. Apart from a glib suggestion that any attempt to\nprevent the work would have created a violent situation endangering the\nsafety of the Ram Janam Bhumi-Babri Masjid structure itself, nothing is\nindicated as to what was sought to be done at all to prevent constructional\nmaterial coming in. There is no mention in any of the affidavits of any of E\nthe officers as to what reasonable measures the Government took to\nprevent the inflow of constructional material such as large quantities of\ncement, mortar, sand, constructional equipment, water-tankers etc. that\nwere necessary for the work. The report of the Expert Committee has\nindicated that constructional machinery was indispensable having regard to F\nthe nature and magnitude of the work carried out. While it is\nunderstandable that the prevention of the ga

In [11]:
prompt_style = '''<s>[INST] <<SYS>>
You are a helpful assistant specialized in Indian law.
<</SYS>>

Context:
{}

Question:
{}
[/INST]
{}
'''

In [12]:
EOS_TOKEN = '</s>'

In [13]:
def preprocess_data(sample):
    context = sample["context"]
    question = sample["question"]
    answer = sample["response"]

    text = prompt_style.format(context, question, answer) + EOS_TOKEN

    return {"texts": text}

In [14]:
def formatting_prompts_func(examples):
    contexts  = examples["context"]
    questions = examples["question"]
    responses = examples["response"]
    texts = []

    # we use zip as whlist formatting in Trainer it recieve a batch which needs to be iterated and appeneded to.
    for context, question, response in zip(contexts, questions, responses):
        text = prompt_style.format(context, question, response) + EOS_TOKEN
        texts.append(text)

    return texts

In [33]:
preprocess_data(law_dataset[5])

{'texts': "<s>[INST] <<SYS>>\nYou are a helpful assistant specialized in Indian law.\n<</SYS>>\n\nContext:\nJunior Engineers, who were not covered\nunder the provisions of Rule\nValidity of the aforesaid government\norder dated   was again\nquestioned before this Court in writ\npetition no.  of   Vijay Kumar\nothers vs. State of U.P. and others and\nvide judgment and order dated\nthis Court set-aside the Government\nOrder dated   and provided that\npromotions shall be made strictly in\naccordance with the   Rules at the\nearliest.\nAgainst the above said judgment\nand order of the High Court, Diploma\nHolder Engineers Sangh filed Special\nLeave Petition No. CC   of\nbefore the Apex Court, which was\nconverted into Civil Appeal No.  of\nInitially, an interim order was\ngranted by the Honble Supreme Court\nproviding that any promotion made would\nbe subject to the outcome of the Special\nLeave Petition. But this SLP which was\nconverted into Civil Appeal filed before\nthe Honble Supreme 

In [16]:
from datasets import Dataset
law_dataset = Dataset.from_list(law_dataset)

finetune_dataset = law_dataset.map(preprocess_data)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [17]:
finetune_dataset.column_names

['context', 'question', 'response', 'texts']

In [18]:
finetune_dataset = finetune_dataset.remove_columns(["context", "question", "response"])

In [19]:
finetune_dataset.column_names

['texts']

In [20]:
# Loading the model

In [21]:
model_name = 'meta-llama/Llama-2-7b-chat-hf'
dataset_name = 'Prarabdha/indian-legal-supervised-fine-tuning-data'

In [22]:
load_in_4bit = True
dtype = None
max_sequence_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    load_in_4bit = load_in_4bit,
    dtype = dtype,
    max_seq_length = max_sequence_length,
    use_gradient_checkpointing='unsloth',
    random_state = 3407
)

==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [23]:
model_lora = FastLanguageModel.get_peft_model(
    model = model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout=0.0,
    bias='none',
    use_gradient_checkpointing='unsloth',
    random_state=3407

)

Unsloth 2026.1.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [24]:
from trl import SFTTrainer

In [25]:
from unsloth import is_bfloat16_supported

In [30]:
trainer = SFTTrainer(
    model = model_lora,
    tokenizer = tokenizer,
    train_dataset = law_dataset,
    formatting_func = formatting_prompts_func,
    max_seq_length = max_sequence_length,
    dataset_num_proc = 1,
    packing = False,

    args = TrainingArguments(
        output_dir = "outputs",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        learning_rate = 2e-4,
        num_train_epochs = 1,
        max_steps = 300,
        warmup_steps = 5,
        fp16 = not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps = 10,
        weight_decay = 0.1,
        optim = "adamw_8bit",
        lr_scheduler_type = 'linear',
        seed = 3407
    ),


)

Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [29]:
import wandb
wnb_token = userdata.get("WAND_TOKEN")
wandb.login(key=wnb_token)
run = wandb.init(
    project='Fine-tune-Llama-2-7b-chat-on-Law-Dataset',
    job_type="training",
    anonymous="allow"
)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jainixshah (jainixshah-d-y-patil-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING The anonymous setting has no effect and will be removed in a future version.


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


In [31]:
trainer_stas = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 2 | Total steps = 300
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 39,976,960 of 6,778,392,576 (0.59% trained)


Step,Training Loss
10,2.293500
20,1.934400
30,1.788500
40,1.756500
50,1.873400
60,1.764000
70,1.893100
80,1.710900
90,1.716500
100,1.666500


train/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
train/grad_norm,▄▁▅▄▂▂▂▂▂▂▄▂▁▁▁▃▁▁▄█▆▃▃▃▄▂▅█▇▆
train/learning_rate,███▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁
train/loss,█▅▄▃▄▄▅▃▃▃▃▃▃▂▂▃▂▃▃▂▃▃▃▅▃▃▂▃▂▁
total_flos,3.5879936496328704e+16
train/epoch,1.2
train/global_step,300
train/grad_norm,0.36278
train/learning_rate,0.0
train/loss,1.4653


In [32]:
wandb.finish()

In [37]:
question = 'What measures did the government take to prevent the inflow of construction materials to the site?'

context = ''''The next question is whether these activities were carried on by a D\ncongregation of Sadhus at the site and not by the State Govt. and despite\nGovernments efforts. Apart from a glib suggestion that any attempt to\nprevent the work would have created a violent situation endangering the\nsafety of the Ram Janam Bhumi-Babri Masjid structure itself, nothing is\nindicated as to what was sought to be done at all to prevent constructional\nmaterial coming in.
There is no mention in any of the affidavits of any of E\nthe officers as to what reasonable measures the Government took to\nprevent the inflow of constructional material such as large quantities of\ncement, mortar, sand, constructional equipment, water-tankers etc. that\nwere necessary for the work.
The report of the Expert Committee has\nindicated that constructional machinery was indispensable having regard to F\nthe nature and magnitude of the work carried out. While it is\nunderstandable that the prevention of the gathering of Sadhus might have\ncreated some resentment, it is un-understandable why large quantities of\nbuilding materials were allowed to be brought on the land unless it be-\nand that must be the reasonable presumption- that the Government itself\nwas not too anxious to prevent it. It is not merely positive acts of violation G\nbut also surreptitious and indirect aids to circumvention and violation of the\norders that are equally impermissible.
If reasonable steps are not taken to\nprevent the violation of the orders of the Court, Government cannot be\nheard to say that violation of the orders were at the instance of others. The\npresumption is that the Government intended not to take such preventive\nsteps. In the facts and circumstances of the case, we are unable to persuade H\nA\nB\nc\nD\nE\nF\nG\nH   SUPa,EME COURT REPORTS   SUPP.  S.C.R.\nourselves to the view that the Government was helpless and the situation\nthat had developed was in spite of all reasonable steps taken by the\nGovernment.
Indeed there is no indication that the Government bestirred\nitself to take any steps, reasonable or otherwise, to prevent large scale\nbuilding material getting into the site. The Chief Minister having given a\nsolemn assurance to the National Integration Council and permitted the\nterms of that assurance to be incorporated at his own undertaking tq this\nCourt and allowed an order to be passed in those terms cannot absolve\nhimself of the responsibility unless he placed before the Court sufficient'''

FastLanguageModel.for_inference(model_lora)
inputs = tokenizer(prompt_style.format(context, question , ""), return_tensors = "pt").to("cuda")

outputs = model_lora.generate(
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 3000,
    use_cache = True
)

response = tokenizer.batch_decode(outputs)
print(response)

["<s><s> [INST] <<SYS>>\nYou are a helpful assistant specialized in Indian law.\n<</SYS>>\n\nContext:\n'The next question is whether these activities were carried on by a D\ncongregation of Sadhus at the site and not by the State Govt. and despite\nGovernments efforts. Apart from a glib suggestion that any attempt to\nprevent the work would have created a violent situation endangering the\nsafety of the Ram Janam Bhumi-Babri Masjid structure itself, nothing is\nindicated as to what was sought to be done at all to prevent constructional\nmaterial coming in. \nThere is no mention in any of the affidavits of any of E\nthe officers as to what reasonable measures the Government took to\nprevent the inflow of constructional material such as large quantities of\ncement, mortar, sand, constructional equipment, water-tankers etc. that\nwere necessary for the work. \nThe report of the Expert Committee has\nindicated that constructional machinery was indispensable having regard to F\nthe nature a

In [43]:
parts = str(response).split("[/INST]")
response_only = parts[-1].strip()
print(response_only)

\n\nThe report of the Expert Committee indicates that constructional machinery was indispensable, but no mention was made of the government's efforts to prevent the inflow of constructional materials such as cement, mortar, sand, water tankers, and other materials.\n</s>"]


In [45]:
raw_dataset_response = 'The government did not take any reasonable measures to prevent the inflow of construction materials such as cement, mortar, sand, constructional equipment, and water tankers to the site.'

In [46]:
print(raw_dataset_response)

The government did not take any reasonable measures to prevent the inflow of construction materials such as cement, mortar, sand, constructional equipment, and water tankers to the site.


In [49]:
#before training response:

question = 'What measures did the government take to prevent the inflow of construction materials to the site?'

context = ''''The next question is whether these activities were carried on by a D\ncongregation of Sadhus at the site and not by the State Govt. and despite\nGovernments efforts. Apart from a glib suggestion that any attempt to\nprevent the work would have created a violent situation endangering the\nsafety of the Ram Janam Bhumi-Babri Masjid structure itself, nothing is\nindicated as to what was sought to be done at all to prevent constructional\nmaterial coming in.
There is no mention in any of the affidavits of any of E\nthe officers as to what reasonable measures the Government took to\nprevent the inflow of constructional material such as large quantities of\ncement, mortar, sand, constructional equipment, water-tankers etc. that\nwere necessary for the work.
The report of the Expert Committee has\nindicated that constructional machinery was indispensable having regard to F\nthe nature and magnitude of the work carried out. While it is\nunderstandable that the prevention of the gathering of Sadhus might have\ncreated some resentment, it is un-understandable why large quantities of\nbuilding materials were allowed to be brought on the land unless it be-\nand that must be the reasonable presumption- that the Government itself\nwas not too anxious to prevent it. It is not merely positive acts of violation G\nbut also surreptitious and indirect aids to circumvention and violation of the\norders that are equally impermissible.
If reasonable steps are not taken to\nprevent the violation of the orders of the Court, Government cannot be\nheard to say that violation of the orders were at the instance of others. The\npresumption is that the Government intended not to take such preventive\nsteps. In the facts and circumstances of the case, we are unable to persuade H\nA\nB\nc\nD\nE\nF\nG\nH   SUPa,EME COURT REPORTS   SUPP.  S.C.R.\nourselves to the view that the Government was helpless and the situation\nthat had developed was in spite of all reasonable steps taken by the\nGovernment.
Indeed there is no indication that the Government bestirred\nitself to take any steps, reasonable or otherwise, to prevent large scale\nbuilding material getting into the site. The Chief Minister having given a\nsolemn assurance to the National Integration Council and permitted the\nterms of that assurance to be incorporated at his own undertaking tq this\nCourt and allowed an order to be passed in those terms cannot absolve\nhimself of the responsibility unless he placed before the Court sufficient'''

FastLanguageModel.for_inference(model)
inputs = tokenizer(prompt_style.format(context, question , ""), return_tensors = "pt").to("cuda")

outputs = model.generate(
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 3000,
    use_cache = False
)

response = tokenizer.batch_decode(outputs)
print(response)

["<s><s> [INST] <<SYS>>\nYou are a helpful assistant specialized in Indian law.\n<</SYS>>\n\nContext:\n'The next question is whether these activities were carried on by a D\ncongregation of Sadhus at the site and not by the State Govt. and despite\nGovernments efforts. Apart from a glib suggestion that any attempt to\nprevent the work would have created a violent situation endangering the\nsafety of the Ram Janam Bhumi-Babri Masjid structure itself, nothing is\nindicated as to what was sought to be done at all to prevent constructional\nmaterial coming in. \nThere is no mention in any of the affidavits of any of E\nthe officers as to what reasonable measures the Government took to\nprevent the inflow of constructional material such as large quantities of\ncement, mortar, sand, constructional equipment, water-tankers etc. that\nwere necessary for the work. \nThe report of the Expert Committee has\nindicated that constructional machinery was indispensable having regard to F\nthe nature a

In [50]:
parts = str(response).split("[/INST]")
response_only = parts[-1].strip()
print(response_only)

\n\nThe report of the Expert Committee indicated that constructional machinery was indispensable due to the nature and magnitude of the work carried out. However, the government did not take reasonable measures to prevent the inflow of constructional materials such as cement, mortar, sand, water-tankers, and other equipment necessary for the work. No mention was made in any of the affidavits of any measures taken to prevent the inflow of these materials.\n</s>"]
